# Hello

# Problem statement and hypothesis

Every year West Nile Virus is a growing concern and causes death in Chicago and nearby suburban areas. The virus is found in infected birds but is passed on to humans by mosquitoes that bit the birds and became infected. So, the project is to use the data where they have found West Nile Virus in mosquitoes in the past to predict where and when West Nile Virus will be present in the future. For more details please go to https://www.kaggle.com/c/predict-west-nile-virus.


# Description of your data set and how it was obtained

The data is provided by Kaggle competition. The data provides the location, mosquito species, weather, spraying (Chicago sprays anti-mosquito chemical in certain areas) data, and whether the virus was found in the mosquitoes. The datasets are spray.csv, weather.csv, train.csv, test.csv. Refer to https://www.kaggle.com/c/predict-west-nile-virus/data for more details (“noaa_weather_qclcd_documentation” provides more details regarding the weather data). 

Basically 
- Train.csv provides data of when and where they trapped mosquitoes (Chicago trap mosquitoes around the city to test for virus), the species of mosquitoes that were trapped, and if they have West Nile Virus. 
- Weather.csv provides a bunch of weather info (temperature, humidity, wind speed, etc.) from 2 weather station. 
- Spray.csv provides when and where they sprayed anti-mosquitoes chemical.
- Test.csv is a test set, please ignore for now.
- Sample_submission.csv is how to format my data when I submit, so ignore this one too.

So far, I have completed parts of the Data Munging process, and I still need to clean the data further. I have successfully combined the training data with the weather data to add features from the weather data to the training data. I will take you through the steps with the code below.

In [2]:
#Installing Packages
import json
import datetime
import numpy as np
import pandas as pd
from math import *

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [3]:
#Reading the data, data acquired from kaggle competition
spray = pd.read_csv('spray.csv')
weather = pd.read_csv('weather.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sampleSubmission.csv')


In [4]:
sample_submission.head()
sample_submission.WnvPresent.unique()

array([0])

# Data Visualization

Parts of my data visualization is done pn Excel, because it is easier to view on Excel than on an ipython notebook.
Most of data is complete, the concern is the weather data which provides data in different units/formats, and also has missing data in different formats (some are blank, some are replaced by letter "M"). The weather data is also taken from two weather stations (2 locations) which does not provide an accurate weather report of the location that mosquitoes were caught which are spread out across the city. I also need to find a way to create features from the spray data which seems like a stand alone data set at the moment. I am planning to use the distance between the location they trapped the mosquitoes and the sprayed area because spraying should reduce the number of mosquito overall and also mosquito with West Nile Virus.

In [9]:
train.head(5)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


In [7]:
train.WnvPresent.value_counts()

0    9955
1     551
dtype: int64

In [577]:
spray.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [578]:
#some data are null, fill na with median
spray_features = spray.fillna(spray.dropna().median()).values
spray_features

array([['2011-08-29', '6:56:58 PM', 42.3916233333333, -88.0891633333333],
       ['2011-08-29', '6:57:08 PM', 42.3913483333333, -88.0891633333333],
       ['2011-08-29', '6:57:18 PM', 42.3910216666667, -88.0891566666667],
       ..., 
       ['2013-09-05', '8:35:21 PM', 42.006021666666705, -87.8173916666667],
       ['2013-09-05', '8:35:31 PM', 42.0054533333333, -87.8174233333333],
       ['2013-09-05', '8:35:41 PM', 42.004805, -87.81746]], dtype=object)

In [579]:
weather.head()
weather.describe()

,Station,Tmax,Tmin,DewPoint,ResultSpeed,ResultDir
count,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000
mean,1.500000,76.166101,57.810462,53.457880,6.960666,17.494905
std,0.500085,11.461970,10.381939,10.675181,3.587527,10.063609
min,1.000000,41.000000,29.000000,22.000000,0.100000,1.000000
25%,1.000000,69.000000,50.000000,46.000000,4.300000,7.000000
50%,1.500000,78.000000,59.000000,54.000000,6.400000,19.000000
75%,2.000000,85.000000,66.000000,62.000000,9.200000,25.000000
max,2.000000,104.000000,83.000000,75.000000,24.100000,36.000000


In [580]:
#SKIP THIS CELL
#Tried  to use variance to to eliminate redundant data, but was not successful yet
#redundancy was seen when I opened the data in excel
#replacing NA data and removing data with low variance
#weather = pd.read_csv('weather.csv')
#weather_features = weather.fillna(weather.dropna().median()).values

'''
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.9 * (1 - .9)))
sel.fit_transform(weather.values)
weather.head()
'''

'\nfrom sklearn.feature_selection import VarianceThreshold\nsel = VarianceThreshold(threshold=(.9 * (1 - .9)))\nsel.fit_transform(weather.values)\nweather.head()\n'

In [581]:
print train.shape
train['WnvPresent'].unique()

(10506, 12)


array([0, 1])

In [583]:
test.head()
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116293 entries, 0 to 116292
Data columns (total 11 columns):
Id                        116293 non-null int64
Date                      116293 non-null object
Address                   116293 non-null object
Species                   116293 non-null object
Block                     116293 non-null int64
Street                    116293 non-null object
Trap                      116293 non-null object
AddressNumberAndStreet    116293 non-null object
Latitude                  116293 non-null float64
Longitude                 116293 non-null float64
AddressAccuracy           116293 non-null int64
dtypes: float64(2), int64(3), object(6)

# Data Munging

So we have 4 sets of data, 3 which we will be using to create the model (spray, weather, and train). I am planning to first merge
the weather data set into the training dataset to add weather features to the set. Second I am planning to use the distance between
each location and the nearest spray site as another feature to make use of the spray data.

In [584]:
#Create two new columns of zeroes in the weather dataframe
#We will add latitude and longitude of each station to the data set after this
weather = pd.read_csv('weather.csv')
weather['S_lat'], weather['S_long']  = np.zeros(len(weather.Cool.values)), np.zeros(len(weather.Cool.values))
weather.head(20)

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,S_lat,S_long
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,M,0.0,0.00,29.10,29.82,1.7,27,9.2,0,0
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,M,M,0.00,29.18,29.82,2.7,25,9.6,0,0
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,M,0.0,0.00,29.38,30.09,13.0,4,13.4,0,0
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,M,M,0.00,29.44,30.08,13.3,2,13.4,0,0
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,M,0.0,0.00,29.39,30.12,11.7,7,11.9,0,0
5,2,2007-05-03,67,48,58,M,40,50,7,0,...,M,M,0.00,29.46,30.12,12.9,6,13.2,0,0
6,1,2007-05-04,66,49,58,4,41,50,7,0,...,M,0.0,T,29.31,30.05,10.4,8,10.8,0,0
7,2,2007-05-04,78,51,M,M,42,50,M,M,...,M,M,0.00,29.36,30.04,10.1,7,10.4,0,0
8,1,2007-05-05,66,53,60,5,38,49,5,0,...,M,0.0,T,29.40,30.10,11.7,7,12.0,0,0
9,2,2007-05-05,66,54,60,M,39,50,5,0,...,M,M,T,29.46,30.09,11.2,7,11.5,0,0


In [586]:
#Add latitutde and longitude of each station to the dataset, S1 = Station 1, S2 = Station 2
#then map it to the table (station latitude and longitude were provided by Kaggle)
S1_lat = 41.995
S1_long= -87.933
S2_lat= 41.786
S2_long=  -87.752
for vec in range(len(weather.Station.values)):
    if weather.Station[vec] == 1:
        weather.S_lat[vec] = S1_lat
        weather.S_long[vec] = S1_long
    else:
        weather.S_lat[vec] = S2_lat
        weather.S_long[vec] = S2_long

        


In [587]:
# Join station 1 and 2 horizontally
weather_stn1 = weather[weather['Station']==1]
weather_stn2 = weather[weather['Station']==2]
weather_stn1 = weather_stn1.drop('Station', axis=1)
weather_stn2 = weather_stn2.drop('Station', axis=1)
weather = weather_stn1.merge(weather_stn2, on='Date')

In [588]:
#View unique values of each feature if there are less than 10 unique feature

for i in range(len(weather.columns)):
    weather_values = weather.values
    if len(np.unique(weather_values[:,i])) <50:
        print weather[weather.columns[i]].head(1), np.unique(weather_values[:,i])


0    14
Name: Depart_x, dtype: object [' 0' ' 1' ' 2' ' 3' ' 4' ' 5' ' 6' ' 7' ' 8' ' 9' '-1' '-10' '-11' '-12'
 '-13' '-14' '-15' '-16' '-17' '-2' '-3' '-4' '-5' '-6' '-7' '-8' '-9' '10'
 '11' '12' '13' '14' '15' '16' '17' '18' '19' '20' '21' '22' '23']
0    56
Name: WetBulb_x, dtype: object ['32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43' '44' '45' '46'
 '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57' '58' '59' '60' '61'
 '62' '63' '64' '65' '66' '67' '68' '69' '70' '71' '72' '73' '74' '75' '76'
 '77' '78' 'M']
0    0
Name: Heat_x, dtype: object ['0' '1' '10' '11' '12' '13' '14' '15' '16' '17' '18' '19' '2' '20' '21'
 '22' '23' '24' '25' '26' '27' '28' '29' '3' '4' '5' '6' '7' '8' '9']
0     2
Name: Cool_x, dtype: object [' 0' ' 1' ' 2' ' 3' ' 4' ' 5' ' 6' ' 7' ' 8' ' 9' '10' '11' '12' '13' '14'
 '15' '16' '17' '18' '19' '20' '21' '22' '23' '24' '26' '28']
0    0
Name: Depth_x, dtype: object ['0']
0    M
Name: Water1_x, dtype: object ['M']
0    0.0
Name: SnowFall_x

In [592]:
#Water1 is all empty so eliminate the column, also eliminate CodeSum since the other parameters already account for it
#weather = weather.drop(['CodeSum'],1)
train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


In [593]:
#before replacing values
weather.head()

,Date,Tmax_x,Tmin_x,Tavg_x,Depart_x,DewPoint_x,WetBulb_x,Heat_x,Cool_x,Sunrise_x,...,Water1_y,SnowFall_y,PrecipTotal_y,StnPressure_y,SeaLevel_y,ResultSpeed_y,ResultDir_y,AvgSpeed_y,S_lat_y,S_long_y
0,2007-05-01,83,50,67,14,51,56,0,2,0448,...,NaN,NaN,0.00,29.18,29.82,2.7,25,9.6,41.786,-87.752
1,2007-05-02,59,42,51,-3,42,47,14,0,0447,...,NaN,NaN,0.00,29.44,30.08,13.3,2,13.4,41.786,-87.752
2,2007-05-03,66,46,56,2,40,48,9,0,0446,...,NaN,NaN,0.00,29.46,30.12,12.9,6,13.2,41.786,-87.752
3,2007-05-04,66,49,58,4,41,50,7,0,0444,...,NaN,NaN,0.00,29.36,30.04,10.1,7,10.4,41.786,-87.752
4,2007-05-05,66,53,60,5,38,49,5,0,0443,...,NaN,NaN,0,29.46,30.09,11.2,7,11.5,41.786,-87.752


In [594]:
#Replace M, space, and - with NA
weather= weather.replace(['M','-', ' ','  T','-inf'], [np.nan, np.nan, np.nan, 0.0,np.nan])
weather.head()

,Date,Tmax_x,Tmin_x,Tavg_x,Depart_x,DewPoint_x,WetBulb_x,Heat_x,Cool_x,Sunrise_x,...,Water1_y,SnowFall_y,PrecipTotal_y,StnPressure_y,SeaLevel_y,ResultSpeed_y,ResultDir_y,AvgSpeed_y,S_lat_y,S_long_y
0,2007-05-01,83,50,67,14,51,56,0,2,0448,...,NaN,NaN,0.00,29.18,29.82,2.7,25,9.6,41.786,-87.752
1,2007-05-02,59,42,51,-3,42,47,14,0,0447,...,NaN,NaN,0.00,29.44,30.08,13.3,2,13.4,41.786,-87.752
2,2007-05-03,66,46,56,2,40,48,9,0,0446,...,NaN,NaN,0.00,29.46,30.12,12.9,6,13.2,41.786,-87.752
3,2007-05-04,66,49,58,4,41,50,7,0,0444,...,NaN,NaN,0.00,29.36,30.04,10.1,7,10.4,41.786,-87.752
4,2007-05-05,66,53,60,5,38,49,5,0,0443,...,NaN,NaN,0,29.46,30.09,11.2,7,11.5,41.786,-87.752


In [674]:
#dropna
print weather.shape
weather_na = weather.dropna(1)
print weather_na.shape
weather_features = weather.columns

#if we dropna we lose a lot of data ~half
weather_na.head(6)

(1472, 45)
(1472, 25)


,Date,Tmax_x,Tmin_x,Tavg_x,Depart_x,DewPoint_x,Heat_x,Cool_x,Sunrise_x,Sunset_x,...,AvgSpeed_x,S_lat_x,S_long_x,Tmax_y,Tmin_y,DewPoint_y,ResultSpeed_y,ResultDir_y,S_lat_y,S_long_y
0,2007-05-01,83,50,67,14,51,0,2,0448,1849,...,9.2,41.995,-87.933,84,52,51,2.7,25,41.786,-87.752
1,2007-05-02,59,42,51,-3,42,14,0,0447,1850,...,13.4,41.995,-87.933,60,43,42,13.3,2,41.786,-87.752
2,2007-05-03,66,46,56,2,40,9,0,0446,1851,...,11.9,41.995,-87.933,67,48,40,12.9,6,41.786,-87.752
3,2007-05-04,66,49,58,4,41,7,0,0444,1852,...,10.8,41.995,-87.933,78,51,42,10.1,7,41.786,-87.752
4,2007-05-05,66,53,60,5,38,5,0,0443,1853,...,12.0,41.995,-87.933,66,54,39,11.2,7,41.786,-87.752
5,2007-05-06,68,49,59,4,30,6,0,0442,1855,...,15.0,41.995,-87.933,68,52,30,13.8,10,41.786,-87.752


In [596]:
#join weather and training data by date
train_weather = pd.merge(train, weather_na, on = 'Date')
print train_weather.shape
train_weather.head(5)


(10506, 36)


,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,...,AvgSpeed_x,S_lat_x,S_long_x,Tmax_y,Tmin_y,DewPoint_y,ResultSpeed_y,ResultDir_y,S_lat_y,S_long_y
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,...,6.5,41.995,-87.933,88,65,59,5.8,16,41.786,-87.752
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,...,6.5,41.995,-87.933,88,65,59,5.8,16,41.786,-87.752
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,...,6.5,41.995,-87.933,88,65,59,5.8,16,41.786,-87.752
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,...,6.5,41.995,-87.933,88,65,59,5.8,16,41.786,-87.752
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,...,6.5,41.995,-87.933,88,65,59,5.8,16,41.786,-87.752


In [599]:
#View unique values of each feature
for i in range(len(train_weather.columns)):
    weather_min_values = train_weather.values
    if len(np.unique(weather_min_values[:,i])) <10:
        print train_weather[train_weather.columns[i]].head(1), np.unique(train_weather.values[:,i])


0    CULEX PIPIENS/RESTUANS
Name: Species, dtype: object ['CULEX ERRATICUS' 'CULEX PIPIENS' 'CULEX PIPIENS/RESTUANS'
 'CULEX RESTUANS' 'CULEX SALINARIUS' 'CULEX TARSALIS' 'CULEX TERRITANS']
0    9
Name: AddressAccuracy, dtype: int64 [3 5 8 9]
0    0
Name: WnvPresent, dtype: int64 [0 1]
0    0
Name: Depth_x, dtype: object ['0']
0    0.0
Name: SnowFall_x, dtype: object [0.0 '0.0']
0    41.995
Name: S_lat_x, dtype: float64 [41.995]
0   -87.933
Name: S_long_x, dtype: float64 [-87.933]
0    41.786
Name: S_lat_y, dtype: float64 [41.786]
0   -87.752
Name: S_long_y, dtype: float64 [-87.752]


# Change mosquitos species into numbers

In [600]:
train_weather.head(5)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,...,AvgSpeed_x,S_lat_x,S_long_x,Tmax_y,Tmin_y,DewPoint_y,ResultSpeed_y,ResultDir_y,S_lat_y,S_long_y
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,...,6.5,41.995,-87.933,88,65,59,5.8,16,41.786,-87.752
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,...,6.5,41.995,-87.933,88,65,59,5.8,16,41.786,-87.752
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,...,6.5,41.995,-87.933,88,65,59,5.8,16,41.786,-87.752
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,...,6.5,41.995,-87.933,88,65,59,5.8,16,41.786,-87.752
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,...,6.5,41.995,-87.933,88,65,59,5.8,16,41.786,-87.752


In [601]:
train_weather.columns

Index([u'Date', u'Address', u'Species', u'Block', u'Street', u'Trap', u'AddressNumberAndStreet', u'Latitude', u'Longitude', u'AddressAccuracy', u'NumMosquitos', u'WnvPresent', u'Tmax_x', u'Tmin_x', u'Tavg_x', u'Depart_x', u'DewPoint_x', u'Heat_x', u'Cool_x', u'Sunrise_x', u'Sunset_x', u'Depth_x', u'SnowFall_x', u'PrecipTotal_x', u'ResultSpeed_x', u'ResultDir_x', u'AvgSpeed_x', u'S_lat_x', u'S_long_x', u'Tmax_y', u'Tmin_y', u'DewPoint_y', u'ResultSpeed_y', u'ResultDir_y', u'S_lat_y', u'S_long_y'], dtype='object')

In [614]:
#take out address features because we already have latitudes and longitudes
train_weather2= train_weather.drop(train_weather.columns[[1,4,5,6]], axis=1)
train_weather2.head(3)

,Date,Species,Block,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Tmax_x,Tmin_x,...,AvgSpeed_x,S_lat_x,S_long_x,Tmax_y,Tmin_y,DewPoint_y,ResultSpeed_y,ResultDir_y,S_lat_y,S_long_y
0,2007-05-29,CULEX PIPIENS/RESTUANS,41,41.954690,-87.800991,9,1,0,88,60,...,6.5,41.995,-87.933,88,65,59,5.8,16,41.786,-87.752
1,2007-05-29,CULEX RESTUANS,41,41.954690,-87.800991,9,1,0,88,60,...,6.5,41.995,-87.933,88,65,59,5.8,16,41.786,-87.752
2,2007-05-29,CULEX RESTUANS,62,41.994991,-87.769279,9,1,0,88,60,...,6.5,41.995,-87.933,88,65,59,5.8,16,41.786,-87.752


In [604]:
#create dummy variables for Species data
train_species = pd.get_dummies(train_weather2.Species, prefix='Species')
#train_species['NumMosquitos'] = train_weather2.NumMosquitos
train_species.head(5)

,Species_CULEX ERRATICUS,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS
0,0,0,1,0,0,0,0
1,0,0,0,1,0,0,0
2,0,0,0,1,0,0,0
3,0,0,1,0,0,0,0
4,0,0,0,1,0,0,0


In [615]:
np.sum(train_species[train_species.index == 1].values)

1.0

In [673]:
#3rd column corelates with 2nd and 4th column, if value of column 3 is 1 then column 2 and 4 are 1

for r in train_species.index:
    if train_species[train_species.columns[2]][r] > 0:
        train_species[train_species.columns[1]][r] = train_species[train_species.columns[2]][r]
        train_species[train_species.columns[3]][r] = train_species[train_species.columns[2]][r]

#drop column 3
train_species= train_species.drop('Species_CULEX PIPIENS/RESTUANS',1)

train_species.head(5)

        
        


ValueError: labels ['Species_CULEX PIPIENS/RESTUANS'] not contained in axis

In [607]:
#concategate species table back to train_weather table
train_final = pd.concat([train_weather2, train_species], axis =1)
print train_weather_min.shape
print train_species.shape
train_final.shape

(0, 36)
(10506, 6)


(10506, 38)

In [618]:
train_final2 = train_final.drop(['Species'], 1)

# Spray Data Feature Extraction

In [ ]:
#Insert Spray Data, remove time data
print spray.shape
spray_drop = spray.drop('Time',1)
spray_drop.columns = ['Date','Latitude_Spray','Longitude_Spray']
spray_drop.head()


In [490]:
spray_drop.Date.head()

0    2011-08-29
1    2011-08-29
2    2011-08-29
3    2011-08-29
4    2011-08-29
Name: Date, dtype: object

In [491]:
#merge spray data_drop data into train data
#train_spray = pd.merge(train_final2, spray_drop, how = 'inner', on = 'Date')
import datetime as dt
for i in range(len(spray_drop.Date)):
    spray_drop.Date[i] = dt.datetime.strptime(spray_drop.Date[i], "%Y-%m-%d")


In [492]:

spray_drop.columns = (['Spray_Date','Latitude_Spray', 'Longitude_Spray'])
spray_drop.head(3)

,Spray_Date,Latitude_Spray,Longitude_Spray
0,2011-08-29 00:00:00,42.391623,-88.089163
1,2011-08-29 00:00:00,42.391348,-88.089163
2,2011-08-29 00:00:00,42.391022,-88.089157


In [672]:
# format training date string
import datetime as dt
for i in range(len(train_final2.Date)):
    train_final2.Date[i] = dt.datetime.strptime( train_final2.Date[i],"%Y-%m-%d")


#for j in range(len(train_final2.Date)):
#    train_final2.Date[j+1] = train_final2.Date[j+1][2:4]+ ' ' + train_final2.Date[j+1][5:7]+ ' ' + train_final2.Date[j+1][8:10]

In [619]:
train_final2.head(2)

,Date,Block,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Tmax_x,Tmin_x,Tavg_x,...,ResultSpeed_y,ResultDir_y,S_lat_y,S_long_y,Species_CULEX ERRATICUS,Species_CULEX PIPIENS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS
0,2007-05-29,41,41.95469,-87.800991,9,1,0,88,60,74,...,5.8,16,41.786,-87.752,0,1,1,0,0,0
1,2007-05-29,41,41.95469,-87.800991,9,1,0,88,60,74,...,5.8,16,41.786,-87.752,0,0,1,0,0,0


In [620]:
train_spray = pd.concat([train_final2, spray_drop], axis =1, join = 'inner')

In [621]:
print train_final2.shape
print spray_drop.shape
train_spray.shape

(10506, 37)
(14835, 3)


(10506, 40)

In [571]:
train_date = train_final2[['Date','Latitude','Longitude']]
train_date.head()

,Date,Latitude,Longitude
1,2007-05-29,41.954690,-87.800991
3,2007-05-29,41.954690,-87.800991
5,2007-05-29,41.994991,-87.769279
7,2007-05-29,41.974089,-87.824812
9,2007-05-29,41.974089,-87.824812


In [ ]:
import datetime as dt
for i in range(len(train_final3.Date)):
    train_final3.Date[i] = dt.datetime.strptime( train_final3.Date[i],"%Y-%m-%d")

In [432]:

#mean distance between the locations that were sprayed in the same day
"""
min_distance = []
train_date['Distance'] = np.zeros(len(train_date.index))
for d in train_date.index:
    print d
    all_distance = []
    for i in spray_drop.index:
        if train_date.Date[d] == spray_drop.Spray_Date[i]:
            lat_diff = train_date.Latitude[d]-spray_drop.Latitude_Spray[i]
            long_diff = train_date.Longitude[d]-spray_drop.Longitude_Spray[i]
            d1 = 2*3959*acos(sqrt(sin(lat_diff/2)**2+cos(train_date.Latitude[d])*cos(spray_drop.Latitude_Spray[i])*sin(long_diff/2)**2))
            all_distance.append(d1)
    min_distance.append(np.mean(all_distance))
"""   
len(np.unique(min_distance))


9971

In [419]:
spray_drop.head()
#train_date.head(3)

,Spray_Date,Latitude_Spray,Longitude_Spray
0,2011-08-29 00:00:00,42.391623,-88.089163
1,2011-08-29 00:00:00,42.391348,-88.089163
2,2011-08-29 00:00:00,42.391022,-88.089157
3,2011-08-29 00:00:00,42.390637,-88.089158
4,2011-08-29 00:00:00,42.390410,-88.088858


In [425]:
#Create time difference between the trap and the spray that was sprayed in the closest location
"""
date_min= []
for d in train_date.index:
    all_distance2 = []
    print d
    for i in spray_drop.index:
        lat_diff = train_date.Latitude[d]-spray_drop.Latitude_Spray[i]
        long_diff = train_date.Longitude[d]-spray_drop.Longitude_Spray[i]
        d1 = 2*3959*acos(sqrt(sin(lat_diff/2)**2+cos(train_date.Latitude[d])*cos(spray_drop.Latitude_Spray[i])*sin(long_diff/2)**2))
        all_distance2.append(d1)
    index = all_distance2.index(min(all_distance2))
    date_min.append(train_date.Date[d] - spray_drop.Spray_Date[index])   
"""
date_min[1:5]






[datetime.timedelta(-1553),
 datetime.timedelta(-1553),
 datetime.timedelta(-1553),
 datetime.timedelta(-1553)]

# Plugging Data into the Model

In [622]:
train_final2.columns

Index([u'Date', u'Block', u'Latitude', u'Longitude', u'AddressAccuracy', u'NumMosquitos', u'WnvPresent', u'Tmax_x', u'Tmin_x', u'Tavg_x', u'Depart_x', u'DewPoint_x', u'Heat_x', u'Cool_x', u'Sunrise_x', u'Sunset_x', u'Depth_x', u'SnowFall_x', u'PrecipTotal_x', u'ResultSpeed_x', u'ResultDir_x', u'AvgSpeed_x', u'S_lat_x', u'S_long_x', u'Tmax_y', u'Tmin_y', u'DewPoint_y', u'ResultSpeed_y', u'ResultDir_y', u'S_lat_y', u'S_long_y', u'Species_CULEX ERRATICUS', u'Species_CULEX PIPIENS', u'Species_CULEX RESTUANS', u'Species_CULEX SALINARIUS', u'Species_CULEX TARSALIS', u'Species_CULEX TERRITANS'], dtype='object')

In [639]:
train_final3 = train_final2.drop(train_final2.columns[[5,6]],1)
features_train = train_final3
target_train = train_final2['WnvPresent']
# format training date string
print train_final3.shape
train_final3.head(3)

(10506, 35)


,Date,Block,Latitude,Longitude,AddressAccuracy,Tmax_x,Tmin_x,Tavg_x,Depart_x,DewPoint_x,...,ResultSpeed_y,ResultDir_y,S_lat_y,S_long_y,Species_CULEX ERRATICUS,Species_CULEX PIPIENS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS
0,2007-05-29,41,41.954690,-87.800991,9,88,60,74,10,58,...,5.8,16,41.786,-87.752,0,1,1,0,0,0
1,2007-05-29,41,41.954690,-87.800991,9,88,60,74,10,58,...,5.8,16,41.786,-87.752,0,0,1,0,0,0
2,2007-05-29,62,41.994991,-87.769279,9,88,60,74,10,58,...,5.8,16,41.786,-87.752,0,0,1,0,0,0


In [640]:
features_train = features_train.drop('Date',1)

In [641]:
#separate train test data
from sklearn.cross_validation import train_test_split, cross_val_score
x_train, x_test, y_train, y_test = train_test_split(features_train, target_train, test_size=0.3, random_state=10)






In [642]:

#Use decision tree
from sklearn import ensemble, preprocessing
clf = ensemble.RandomForestClassifier(n_jobs=-1, n_estimators=1000, min_samples_split=1)
clf.fit(x_train, y_train)


RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
            min_samples_split=1, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0)

In [739]:
#Try Knn
from sklearn import neighbors, datasets, feature_selection
from sklearn.cross_validation import cross_val_score
knn = neighbors.KNeighborsClassifier(50, weights='uniform')
knn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_neighbors=50, p=2, weights='uniform')

In [740]:
#cross validation to evaluate model
scores = cross_val_score(knn, features_train, target_train, cv=5)
scores.mean()

0.94755381312238163

In [748]:
#feature importances
pd.DataFrame({'feature':features_train.columns, 'importance':clf.feature_importances_})

,feature,importance
0,Block,0.168124
1,Latitude,0.203672
2,Longitude,0.221867
3,AddressAccuracy,0.051894
4,Tmax_x,0.010242
5,Tmin_x,0.010607
6,Tavg_x,0.010291
7,Depart_x,0.009784
8,DewPoint_x,0.009858
9,Heat_x,0.000712


# Clean Testing Data

In [719]:
test.head()

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


In [720]:
test_weather = pd.merge(test, weather_na, how = 'inner', on = 'Date')
print test_weather.shape
print test.shape

(116293, 35)
(116293, 11)


In [722]:

#reindex the training and weather table data to have only the data of the closest weather station.
test_weather_min = pd.DataFrame(test_weather, index = test_index)
#view reindexed data, the data frame should give us the data of where and when they caught the mosquitoes, whether it has the virus, 
#and the weather information of the closest weather station


In [723]:
#create dummy variables for Species data
test_species = pd.get_dummies(test_weather.Species, prefix = 'Species')
test_species.head()

,Species_CULEX ERRATICUS,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,Species_UNSPECIFIED CULEX
0,0,0,1,0,0,0,0,0
1,0,0,0,1,0,0,0,0
2,0,1,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,1,0


In [724]:
#2d column corelates with 1st and 3rd column, if value of column 2 is 1 then column 1 and 3 are 1
for r in test_species.index:
    if test_species[test_species.columns[1]][r] == 1:
        test_species[test_species.columns[0]][r] = 1
        test_species[test_species.columns[2]][r] = 1
    else:
         1+1
#drop column 3
test_species = test_species.drop('Species_CULEX PIPIENS/RESTUANS',1)
test_species = test_species.drop('Species_UNSPECIFIED CULEX',1)
test_species['Species_CULEX ERRATICUS'] = np.zeros(len(test_species.index))

test_species.head(5)
        
        
        

,Species_CULEX ERRATICUS,Species_CULEX PIPIENS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS
0,0,0,0,0,0,0
1,0,0,1,0,0,0
2,0,1,0,0,0,0
3,0,0,0,1,0,0
4,0,0,0,0,0,1


In [725]:
#combine species back into main dataframe
test_final = pd.concat([test_weather, test_species], axis=1)

In [726]:
test_final.head(1)

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,...,ResultSpeed_y,ResultDir_y,S_lat_y,S_long_y,Species_CULEX ERRATICUS,Species_CULEX PIPIENS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,...,9.4,18,41.786,-87.752,0,0,0,0,0,0


In [727]:
#create new test data array with date, latitude , and longitude
test_date = test_final[['Date','Latitude','Longitude']]
test_date.head(2)

,Date,Latitude,Longitude
0,2008-06-11,41.95469,-87.800991
1,2008-06-11,41.95469,-87.800991


In [728]:
#convert date to datetime
#import datetime as dt
#for i in range(len(test_date.Date)):
#    test_date.Date[i] = dt.datetime.strptime(test_date.Date[i], "%Y-%m-%d")

In [729]:
#mean distance between the locations that were sprayed in the same day
'''
min_distance2 = []
test_date['Distance'] = np.zeros(len(test_date.index))
for d in test_date.index:
    print d
    all_distance = []
    for i in spray_drop.index:
        if test_date.Date[d] == spray_drop.Spray_Date[i]:
            lat_diff = test_date.Latitude[d]-spray_drop.Latitude_Spray[i]
            long_diff = test_date.Longitude[d]-spray_drop.Longitude_Spray[i]
            d1 = 2*3959*acos(sqrt(sin(lat_diff/2)**2+cos(test_date.Latitude[d])*cos(spray_drop.Latitude_Spray[i])*sin(long_diff/2)**2))
            all_distance2.append(d1)
    min_distance2.append(np.mean(all_distance))
'''

"\nmin_distance2 = []\ntest_date['Distance'] = np.zeros(len(test_date.index))\nfor d in test_date.index:\n    print d\n    all_distance = []\n    for i in spray_drop.index:\n        if test_date.Date[d] == spray_drop.Spray_Date[i]:\n            lat_diff = test_date.Latitude[d]-spray_drop.Latitude_Spray[i]\n            long_diff = test_date.Longitude[d]-spray_drop.Longitude_Spray[i]\n            d1 = 2*3959*acos(sqrt(sin(lat_diff/2)**2+cos(test_date.Latitude[d])*cos(spray_drop.Latitude_Spray[i])*sin(long_diff/2)**2))\n            all_distance2.append(d1)\n    min_distance2.append(np.mean(all_distance))\n"

In [730]:
#Create time difference between the trap and the spray that was sprayed in the closest location
'''
date_min= []
for d in train_date.index:
    all_distance2 = []
    print d
    for i in spray_drop.index:
        lat_diff = train_date.Latitude[d]-spray_drop.Latitude_Spray[i]
        long_diff = train_date.Longitude[d]-spray_drop.Longitude_Spray[i]
        d1 = 2*3959*acos(sqrt(sin(lat_diff/2)**2+cos(train_date.Latitude[d])*cos(spray_drop.Latitude_Spray[i])*sin(long_diff/2)**2))
        all_distance2.append(d1)
    index = all_distance2.index(min(all_distance2))
    date_min.append(train_date.Date[d] - spray_drop.Spray_Date[index])   
'''      

'\ndate_min= []\nfor d in train_date.index:\n    all_distance2 = []\n    print d\n    for i in spray_drop.index:\n        lat_diff = train_date.Latitude[d]-spray_drop.Latitude_Spray[i]\n        long_diff = train_date.Longitude[d]-spray_drop.Longitude_Spray[i]\n        d1 = 2*3959*acos(sqrt(sin(lat_diff/2)**2+cos(train_date.Latitude[d])*cos(spray_drop.Latitude_Spray[i])*sin(long_diff/2)**2))\n        all_distance2.append(d1)\n    index = all_distance2.index(min(all_distance2))\n    date_min.append(train_date.Date[d] - spray_drop.Spray_Date[index])   \n'

In [731]:
test_final.columns

Index([u'Id', u'Date', u'Address', u'Species', u'Block', u'Street', u'Trap', u'AddressNumberAndStreet', u'Latitude', u'Longitude', u'AddressAccuracy', u'Tmax_x', u'Tmin_x', u'Tavg_x', u'Depart_x', u'DewPoint_x', u'Heat_x', u'Cool_x', u'Sunrise_x', u'Sunset_x', u'Depth_x', u'SnowFall_x', u'PrecipTotal_x', u'ResultSpeed_x', u'ResultDir_x', u'AvgSpeed_x', u'S_lat_x', u'S_long_x', u'Tmax_y', u'Tmin_y', u'DewPoint_y', u'ResultSpeed_y', u'ResultDir_y', u'S_lat_y', u'S_long_y', u'Species_CULEX ERRATICUS', u'Species_CULEX PIPIENS', u'Species_CULEX RESTUANS', u'Species_CULEX SALINARIUS', u'Species_CULEX TARSALIS', u'Species_CULEX TERRITANS'], dtype='object')

In [732]:
#drop unnecessary columns
test_final2 = test_final.drop(test_final.columns[[2,3,5,6,7]],1)
print test_final2.shape
test_final2.head()

(116293, 36)


,Id,Date,Block,Latitude,Longitude,AddressAccuracy,Tmax_x,Tmin_x,Tavg_x,Depart_x,...,ResultSpeed_y,ResultDir_y,S_lat_y,S_long_y,Species_CULEX ERRATICUS,Species_CULEX PIPIENS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS
0,1,2008-06-11,41,41.95469,-87.800991,9,86,61,74,7,...,9.4,18,41.786,-87.752,0,0,0,0,0,0
1,2,2008-06-11,41,41.95469,-87.800991,9,86,61,74,7,...,9.4,18,41.786,-87.752,0,0,1,0,0,0
2,3,2008-06-11,41,41.95469,-87.800991,9,86,61,74,7,...,9.4,18,41.786,-87.752,0,1,0,0,0,0
3,4,2008-06-11,41,41.95469,-87.800991,9,86,61,74,7,...,9.4,18,41.786,-87.752,0,0,0,1,0,0
4,5,2008-06-11,41,41.95469,-87.800991,9,86,61,74,7,...,9.4,18,41.786,-87.752,0,0,0,0,0,1


In [733]:
#create features test
features_test = test_final[test_final2.columns[2:]]
features_test.head(5)

,Block,Latitude,Longitude,AddressAccuracy,Tmax_x,Tmin_x,Tavg_x,Depart_x,DewPoint_x,Heat_x,...,ResultSpeed_y,ResultDir_y,S_lat_y,S_long_y,Species_CULEX ERRATICUS,Species_CULEX PIPIENS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS
0,41,41.95469,-87.800991,9,86,61,74,7,56,0,...,9.4,18,41.786,-87.752,0,0,0,0,0,0
1,41,41.95469,-87.800991,9,86,61,74,7,56,0,...,9.4,18,41.786,-87.752,0,0,1,0,0,0
2,41,41.95469,-87.800991,9,86,61,74,7,56,0,...,9.4,18,41.786,-87.752,0,1,0,0,0,0
3,41,41.95469,-87.800991,9,86,61,74,7,56,0,...,9.4,18,41.786,-87.752,0,0,0,1,0,0
4,41,41.95469,-87.800991,9,86,61,74,7,56,0,...,9.4,18,41.786,-87.752,0,0,0,0,0,1


In [742]:
target_predicted = knn.predict_proba(features_test)[:,1]
target_predicted

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [743]:
#Create Submission file
submission = test_final2.drop(test_final2.columns[1:],1)
submission['WnvPresent'] = target_predicted
submission.head(5)

,Id,WnvPresent
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [744]:
print submission.head()
print sample_submission.head()

   Id  WnvPresent
0   1           0
1   2           0
2   3           0
3   4           0
4   5           0
   Id  WnvPresent
0   1           0
1   2           0
2   3           0
3   4           0
4   5           0


In [745]:
#Create Submission File
submission.to_csv('/home/jerd/Desktop/submission.csv',columns=['Id','WnvPresent'], index = 'False')

In [746]:
#Check the Submission Value
submission.WnvPresent.unique()

array([ 0.  ,  0.02,  0.06,  0.04,  0.1 ,  0.08,  0.14,  0.12,  0.18,
        0.16,  0.22,  0.2 ,  0.24,  0.26,  0.28,  0.3 ,  0.32])

# Conclusion

Random Forest performs the best out of the 3 algorithms, Random Forest, Logistic Regression, Knn Classifier. Also the more data you have the better the algorithm perform. Since these classifiers can take in correlated data you should appropriately provide as much data as possible. 

# Appendix (Codes that were used but got removed to improve classifier performance)

In [675]:
#Since each date has two weather data c,alculate the distance between the location they captured the mosquito and the 
#weather station and pick the closest weather station.
#Return the index of rows with the closest weather station
#NOT IN USE BECAUSE I CHANGE TO USING BOTH WEATHER DATA
'''
train_index = []

for i in range(0,len(train_weather),2):
    lat_diff = train_weather.S_lat[i]-train_weather.Latitude[i]
    long_diff = train_weather.S_long[i]-train_weather.Longitude[i]
    
    lat_diff2 = train_weather.S_lat[i+1]-train_weather.Latitude[i+1]
    long_diff2 = train_weather.S_long[i+1]-train_weather.Longitude[i+1]
    
    d1 = 2*3959*acos(sqrt(sin(lat_diff/2)**2+cos(train_weather.S_lat[i])*cos(train_weather.Latitude[i])*sin(long_diff/2)**2))
    d2 = 2*3959*acos(sqrt(sin(lat_diff2/2)**2+cos(train_weather.S_lat[i+1])*cos(train_weather.Latitude[i+1])*sin(long_diff2/2)**2))
    
    if d1 >= d2:
        train_index.append(i+1)
    else:
        train_index.append(i)
'''  
    
    
    
    

'\ntrain_index = []\n\nfor i in range(0,len(train_weather),2):\n    lat_diff = train_weather.S_lat[i]-train_weather.Latitude[i]\n    long_diff = train_weather.S_long[i]-train_weather.Longitude[i]\n    \n    lat_diff2 = train_weather.S_lat[i+1]-train_weather.Latitude[i+1]\n    long_diff2 = train_weather.S_long[i+1]-train_weather.Longitude[i+1]\n    \n    d1 = 2*3959*acos(sqrt(sin(lat_diff/2)**2+cos(train_weather.S_lat[i])*cos(train_weather.Latitude[i])*sin(long_diff/2)**2))\n    d2 = 2*3959*acos(sqrt(sin(lat_diff2/2)**2+cos(train_weather.S_lat[i+1])*cos(train_weather.Latitude[i+1])*sin(long_diff2/2)**2))\n    \n    if d1 >= d2:\n        train_index.append(i+1)\n    else:\n        train_index.append(i)\n'

In [676]:
'''
#reindex the training and weather table data to have only the data of the closest weather station.
train_weather_min = pd.DataFrame(train_weather, index = train_index)
#view reindexed data, the data frame should give us the data of where and when they caught the mosquitoes, whether it has the virus, 
#and the weather information of the closest weather station
'''

'\n#reindex the training and weather table data to have only the data of the closest weather station.\ntrain_weather_min = pd.DataFrame(train_weather, index = train_index)\n#view reindexed data, the data frame should give us the data of where and when they caught the mosquitoes, whether it has the virus, \n#and the weather information of the closest weather station\n'

In [721]:
#Pick the closest weather station. Not in used, used for another method that didn't work as well.
#Return the index of rows with the closest weather station
'''
test_index = []

for i in range(0,len(test_weather),2):
    lat_diff = test_weather.S_lat[i]-test_weather.Latitude[i]
    long_diff = test_weather.S_long[i]-test_weather.Longitude[i]
    
    lat_diff2 = test_weather.S_lat[i+1]-test_weather.Latitude[i+1]
    long_diff2 = test_weather.S_long[i+1]-test_weather.Longitude[i+1]
    
    d1 = 2*3959*acos(sqrt(sin(lat_diff/2)**2+cos(test_weather.S_lat[i])*cos(test_weather.Latitude[i])*sin(long_diff/2)**2))
    d2 = 2*3959*acos(sqrt(sin(lat_diff2/2)**2+cos(test_weather.S_lat[i+1])*cos(test_weather.Latitude[i+1])*sin(long_diff2/2)**2))
    
    if d1 >= d2:
        test_index.append(i+1)
    else:
        test_index.append(i)
 '''
        
    
    
    

'\ntest_index = []\n\nfor i in range(0,len(test_weather),2):\n    lat_diff = test_weather.S_lat[i]-test_weather.Latitude[i]\n    long_diff = test_weather.S_long[i]-test_weather.Longitude[i]\n    \n    lat_diff2 = test_weather.S_lat[i+1]-test_weather.Latitude[i+1]\n    long_diff2 = test_weather.S_long[i+1]-test_weather.Longitude[i+1]\n    \n    d1 = 2*3959*acos(sqrt(sin(lat_diff/2)**2+cos(test_weather.S_lat[i])*cos(test_weather.Latitude[i])*sin(long_diff/2)**2))\n    d2 = 2*3959*acos(sqrt(sin(lat_diff2/2)**2+cos(test_weather.S_lat[i+1])*cos(test_weather.Latitude[i+1])*sin(long_diff2/2)**2))\n    \n    if d1 >= d2:\n        test_index.append(i+1)\n    else:\n        test_index.append(i)\n '